# Reading and plotting multiple Datasets

In this notebook we will look at plotting multiple Time series from more than one dataset. Again we need the necessary imports.

In [ ]:
import os
from datetime import datetime
import pandas as pd

from datascience.read import Era5, Era5Land, AscatDataH121, Gldas, read_multiple_ds
from datascience.plot import plot_multiple_gpis, plot_ts

%matplotlib widget

In this example we will look at plotting ERA-5 data with ASCAT data. You first need to create the respective objects.

In [ ]:
era5 = Era5(read_bulk = False)
ascat = AscatDataH121(read_bulk = False)
era5land = Era5Land(read_bulk = False)
gldas = Gldas(read_bulk = False)

And define a lat and lon:

In [ ]:
lat = 48.198905
lon = 16.367182

You can see the gpi locations and spatial resolutions differ by dataset. You can choose between 1 and 4 objects to be plotted with the *plot_mulitple_gpis* function. The closest gpis are highlighted. You can choose how many grid points you want to show per dataset.

In [ ]:
plot_multiple_gpis((lon, lat), ascat, era5, era5land, gldas, k=30)

To read multiple datasets and make them comparable you first need to define a location. As you can see from the plot above, the location of the gpis vary by dataset. To make the results comparable we will choose the closest grid point to the defined location for each dataset with the *grid.find_nearest_gpi()* function. This function returns the closest gpi and the distance to the location.

To keep it simple we will only load ASCAT and Era5 data in this example.

In [ ]:
ascat_gpi, ascat_distance = ascat.grid.find_nearest_gpi(lon, lat)
era5_gpi, era5_distance = era5.grid.find_nearest_gpi(lon, lat)
print(f"Ascat gpi: {ascat_gpi}, distance: {ascat_distance}\nEra5 gpi: {era5_gpi}, distance: {era5_distance}")

If you want to load the directly with the gpi, you first have to get the coordinates of the gpi using the *grid.gpi2lonlat()* function. Lets say you chose the gpi #1227274 from the ascat dataset:

In [ ]:
location = ascat.grid.gpi2lonlat(ascat_gpi)
location

Now you can get the closest Era5-gpi to the ascat gpi:

In [ ]:
era5_gpi, era5_distance = era5.grid.find_nearest_gpi(location[0], location[1])
print(f"Era5 gpi: {era5_gpi}, distance: {era5_distance}")

You can now read the data:

In [ ]:
ascat_data = ascat.read(ascat_gpi)
era5_data = era5.read(era5_gpi)

To merge the data you first need to make sure that the index of the dataframe is in the same format:

In [ ]:
ascat_data.index = ascat_data.index.astype('datetime64[ms]')
era5_data.index = era5_data.index.astype('datetime64[ms]')

You can now merge the data using *pd.merge_asof*. Note that the resulting timestamps will be chosen from the first dataset which is given in the function, which means that information might get lost, or is chosen from the closest available timestamp.

In [ ]:
merged_data = pd.merge_asof(ascat_data,
                       era5_data,
                       left_index=True,
                       right_index=True,
                       tolerance=pd.Timedelta("3h"),
                       direction="nearest")

merged_data

With the *read_multiple_ds* function you can read multiple datasets simultaneously. For the parameter *loc* you can either choose a (lon,lat) location or a gpi, you can choose one or more datasets to be read, but note that you need to choose a reference dataset (either *"ascat", "era5", "era5land"* or *"gldas"*). If you give a gpi for the *loc* parameter the reference dataset needs to be the dataset from which you chose the gpi. 

As a return you get a single dataframe, with the merged data from each dataset. Note that the index for the resulting dataframe is taken from the reference dataset. This means that if the reference dataset has a higher temporal resolution than the other datasets, the according timestamps for the other datasets will be filled with the values from the closest available timestamp. If the reference dataset has a lower temporal resolution some timestamps from the higher resolution data will be lost. Also Note that, if you merge two datasets with the same variable names (era5 and era5land), the according variables will be renamed in the resulting dataframe.

In [ ]:
ts = read_multiple_ds(loc=(lon, lat), ascat=ascat, era5=era5, era5land=era5land, gldas=gldas, ref_ds="ascat")

In [ ]:
ts

You can see which columns are available (*https://codes.ecmwf.int/grib/param-db/?search=sd*):

In [ ]:
ts.columns

You should also mask where soil moisture values are not valid (where the temperature is below freezing and where there is snow):

In [ ]:
not_valid = (ts["stl1_era5"] < 0) | (ts["sd"] > 0)
ts.loc[:,"sm_valid"] = ~not_valid

In [ ]:
ts_valid = ts.loc[ts["sm_valid"]]

Again, you can plot the data from specific columns with the *plot_ts* function

In [ ]:
params = ["surface_soil_moisture", "swvl1_era5", "SoilMoi0_10cm_inst"]
timeperiod = [datetime(2019,1,1), datetime(2021,12,1)]
plot_ts(ts_valid, params, timeperiod, title="Simple Time series")